# Sending and Receiving Packets

## sniff() function
- equivalent to tcpdump and wireshark
- also has built-in wireshark function to parse big and complex packet captures


In [1]:
from scapy.all import *

In [3]:
conf.verb=1
conf.color_theme = RastaTheme()

In [47]:
help(sniff)

Help on function sniff in module scapy.sendrecv:

sniff(count=0, store=True, offline=None, prn=None, lfilter=None, L2socket=None, timeout=None, opened_socket=None, stop_filter=None, iface=None, *arg, **karg)
    Sniff packets and return a list of packets.
    
    Arguments:
    
      count: number of packets to capture. 0 means infinity.
    
      store: whether to store sniffed packets or discard them
    
      prn: function to apply to each packet. If something is returned, it
          is displayed.
    
          Ex: prn = lambda x: x.summary()
    
      filter: BPF filter to apply.
    
      lfilter: Python function applied to each packet to determine if
          further action may be done.
    
          Ex: lfilter = lambda x: x.haslayer(Padding)
    
      offline: PCAP file (or list of PCAP files) to read packets from,
          instead of sniffing them
    
      timeout: stop sniffing after a given time (default: None).
    
      L2socket: use the provided L2socket (

In [4]:
# note: run ping google.com on a terminal to generate icmp packets
pkts = sniff(count=2, filter="icmp")

In [6]:
pkts

<Sniffed: TCP:0 UDP:0 ICMP:2 Other:0>

In [7]:
pkts.summary()

Ether / IP / ICMP 192.168.1.135 > 172.217.2.46 echo-request 0 / Raw
Ether / IP / ICMP 172.217.2.46 > 192.168.1.135 echo-reply 0 / Raw


In [8]:
pkts.show()

0000 Ether / IP / ICMP 192.168.1.135 > 172.217.2.46 echo-request 0 / Raw
0001 Ether / IP / ICMP 172.217.2.46 > 192.168.1.135 echo-reply 0 / Raw


In [9]:
pkts[0].show()

###[ Ethernet ]### 
  dst= 94:10:3e:83:59:9b
  src= 08:00:27:ce:4f:bd
  type= 0x800
###[ IP ]### 
     version= 4
     ihl= 5
     tos= 0x0
     len= 84
     id= 8622
     flags= DF
     frag= 0
     ttl= 64
     proto= icmp
     chksum= 0xa7c4
     src= 192.168.1.135
     dst= 172.217.2.46
     \options\
###[ ICMP ]### 
        type= echo-request
        code= 0
        chksum= 0xcd4f
        id= 0x777a
        seq= 0x1
###[ Raw ]### 
           load= '\xb6\x85\xa0[hP\t\x00\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'



In [10]:
hexdump(pkts[0])

0000  94103E83599B080027CE4FBD08004500 ..>.Y...'.O...E.
0010  005421AE40004001A7C4C0A80187ACD9 .T!.@.@.........
0020  022E0800CD4F777A0001B685A05B6850 .....Owz.....[hP
0030  090008090A0B0C0D0E0F101112131415 ................
0040  161718191A1B1C1D1E1F202122232425 .......... !"#$%
0050  262728292A2B2C2D2E2F303132333435 &'()*+,-./012345
0060  3637                             67


In [11]:
pkts[0].command()

'Ether(dst=\'94:10:3e:83:59:9b\', type=2048, src=\'08:00:27:ce:4f:bd\')/IP(flags=2, version=4, ihl=5, dst=\'172.217.2.46\', options=[], len=84, tos=0, ttl=64, src=\'192.168.1.135\', chksum=42948, id=8622, frag=0, proto=1)/ICMP(ptr=None, ts_ori=None, length=None, code=0, unused=None, seq=1, addr_mask=None, nexthopmtu=None, reserved=None, ts_rx=None, ts_tx=None, chksum=52559, id=30586, type=8, gw=None)/Raw(load=b\'\\xb6\\x85\\xa0[hP\\t\\x00\\x08\\t\\n\\x0b\\x0c\\r\\x0e\\x0f\\x10\\x11\\x12\\x13\\x14\\x15\\x16\\x17\\x18\\x19\\x1a\\x1b\\x1c\\x1d\\x1e\\x1f !"#$%&\\\'()*+,-./01234567\')'

## Sniffing ARP packets 
### Generate ARP packets using arping
1. open a terminal and run $ arping [ip of another vm/gateway]
2. sniff the packets using scapy

In [14]:
# ping a non-existant private ip or gateway or use arping
pkts = sniff(count=5, filter="arp")
pkts

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:5>

In [15]:
pkts.summary()

Ether / ARP who has 192.168.1.100 says 192.168.1.135
Ether / ARP who has 192.168.1.100 says 192.168.1.135
Ether / ARP who has 192.168.1.100 says 192.168.1.135
Ether / ARP who has 192.168.1.100 says 192.168.1.135
Ether / ARP who has 192.168.1.100 says 192.168.1.135


In [16]:
pkts[0].show()

###[ Ethernet ]### 
  dst= ff:ff:ff:ff:ff:ff
  src= 08:00:27:ce:4f:bd
  type= 0x806
###[ ARP ]### 
     hwtype= 0x1
     ptype= 0x800
     hwlen= 6
     plen= 4
     op= who-has
     hwsrc= 08:00:27:ce:4f:bd
     psrc= 192.168.1.135
     hwdst= 00:00:00:00:00:00
     pdst= 192.168.1.100



In [17]:
pkts[0].command()

"Ether(dst='ff:ff:ff:ff:ff:ff', type=2054, src='08:00:27:ce:4f:bd')/ARP(hwsrc='08:00:27:ce:4f:bd', ptype=2048, plen=4, op=1, hwtype=1, hwlen=6, pdst='192.168.1.100', hwdst='00:00:00:00:00:00', psrc='192.168.1.135')"

## Building a Packet
- build ARP packet which has two layers - Ether and ARP

In [12]:
ls(Ether)

dst        : DestMACField         = (None)
src        : SourceMACField       = (None)
type       : XShortEnumField      = (36864)


In [13]:
ls(ARP)

hwtype     : XShortField          = (1)
ptype      : XShortEnumField      = (2048)
hwlen      : ByteField            = (6)
plen       : ByteField            = (4)
op         : ShortEnumField       = (1)
hwsrc      : ARPSourceMACField    = (None)
psrc       : SourceIPField        = (None)
hwdst      : MACField             = ('00:00:00:00:00:00')
pdst       : IPField              = ('0.0.0.0')


## Stacking Layers to build packets

In [18]:
arpPkt = Ether()/ARP()

In [19]:
arpPkt.show()

###[ Ethernet ]### 
  dst= 94:10:3e:83:59:9b
  src= 08:00:27:ce:4f:bd
  type= 0x806
###[ ARP ]### 
     hwtype= 0x1
     ptype= 0x800
     hwlen= 6
     plen= 4
     op= who-has
     hwsrc= 08:00:27:ce:4f:bd
     psrc= 192.168.1.135
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [20]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'

In [21]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 08:00:27:ce:4f:bd
  type= 0x806
###[ ARP ]### 
     hwtype= 0x1
     ptype= 0x800
     hwlen= 6
     plen= 4
     op= who-has
     hwsrc= 08:00:27:ce:4f:bd
     psrc= 192.168.1.135
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [22]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'

In [23]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 08:00:27:ce:4f:bd
  type= 0x806
###[ ARP ]### 
     hwtype= 0x1
     ptype= 0x800
     hwlen= 6
     plen= 4
     op= who-has
     hwsrc= 08:00:27:ce:4f:bd
     psrc= 192.168.1.135
     hwdst= FF:FF:FF:FF:FF:FF
     pdst= 0.0.0.0



## Capture packet using wireshark
- ARP packet is ready to be sent
- Open Wireshark and start capturing traffic

## Send Packets
- two functions to send packets: sendp() and send()
- **sendp()** for sending L2 packets
- **send()** for sending L3 packets (IPv4 or IPv6)

In [24]:
help(send)

Help on function send in module scapy.sendrecv:

send(x, inter=0, loop=0, count=None, verbose=None, realtime=None, return_packets=False, socket=None, *args, **kargs)
    Send packets at layer 3
    send(packets, [inter=0], [loop=0], [count=None], [verbose=conf.verb], [realtime=None], [return_packets=False],
         [socket=None]) -> None



In [25]:
help(sendp)

Help on function sendp in module scapy.sendrecv:

sendp(x, inter=0, loop=0, iface=None, iface_hint=None, count=None, verbose=None, realtime=None, return_packets=False, socket=None, *args, **kargs)
    Send packets at layer 2
    sendp(packets, [inter=0], [loop=0], [iface=None], [iface_hint=None], [count=None], [verbose=conf.verb],
          [realtime=None], [return_packets=False], [socket=None]) -> None



In [26]:
# send the arpPkt - use sendp() because ARP is layer 2 packet
sendp(arpPkt)


Sent 1 packets.


In [27]:
# one liner
sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="00:11:22:aa:bb:cc")/ARP(hwsrc="00:11:22:aa:bb:cc",pdst="192.168.231.2"))


Sent 1 packets.


## Sending HTTP request

In [37]:
httpPkt = IP(dst="www.coloradomesa.edu")/TCP()/"GET /index.html HTTP/1.1\n\n"

In [38]:
bytes(httpPkt)

b'E\x00\x00B\x00\x01\x00\x00@\x06\x8f8\xc0\xa8\x01\x87H\xa6\xe0\xa7\x00\x14\x00P\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\xf24\x00\x00GET /index.html HTTP/1.1\n\n'

In [39]:
hexdump(httpPkt)

0000  450000420001000040068F38C0A80187 E..B....@..8....
0010  48A6E0A7001400500000000000000000 H......P........
0020  50022000F2340000474554202F696E64 P. ..4..GET /ind
0030  65782E68746D6C20485454502F312E31 ex.html HTTP/1.1
0040  0A0A                             ..


In [40]:
httpPkt.show()

###[ IP ]### 
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= tcp
  chksum= None
  src= 192.168.1.135
  dst= Net('www.coloradomesa.edu')
  \options\
###[ TCP ]### 
     sport= ftp_data
     dport= http
     seq= 0
     ack= 0
     dataofs= None
     reserved= 0
     flags= S
     window= 8192
     chksum= None
     urgptr= 0
     options= []
###[ Raw ]### 
        load= 'GET /index.html HTTP/1.1\n\n'



In [42]:
response = sr(httpPkt)

Begin emission:
Finished sending 1 packets.

Received 5 packets, got 1 answers, remaining 0 packets


In [45]:
response[0].show()

0000 IP / TCP 192.168.1.135:ftp_data > 72.166.224.167:http S / Raw ==> IP / TCP 72.166.224.167:http > 192.168.1.135:ftp_data SA / Padding


In [46]:
response[0].summary()

IP / TCP 192.168.1.135:ftp_data > 72.166.224.167:http S / Raw ==> IP / TCP 72.166.224.167:http > 192.168.1.135:ftp_data SA / Padding


## TCP/IP full three-way handshake

### send SYN packet

## Send and Receive Packets
- Send and receive functions are the heart of Scapy
- ** sr() ** send L3 packets and receive answers; returns answered and unanswered packets
- ** sr1() ** returns only one packet that answered the sent packet
- sr() and sr1() functions can only send layer 3 L3 packets (IP, HTTP, etc.)
- ** srp() ** send and receive L2 packets (ARP, Ehternet, 802.3, etc)
- ** srp1() ** send and receive one layer 2 packet

## Three-way handshake


In [ ]:
# Create syn packet
syn = IP(dst='www.google.com')/TCP(dport=80, flags='S')

In [ ]:
syn

In [ ]:
# Receive SYN-ACK packet from google.com
syn_ack = sr1(syn)

In [ ]:
syn_ack

In [ ]:
# Send Get request
getStr = 'GET / HTTP1.1\r\nHost:www.google.com\r\n\r\n'
request = IP(dst='www.google.com') / TCP(dport=80, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr
reply = sr1(request)

In [ ]:
reply

In [ ]:
reply.summary()

In [ ]:
reply.show()

## Stacking Layers
The / operator has been used as a composition operator between two layers. When doing so, the lower layer can have one or more of its defaults fields overloaded according to the upper layer. (You still can give the value you want). A string can be used as a raw layer.
<img src="http://scapy.readthedocs.io/en/latest/_images/fieldsmanagement.png" />

In [23]:
IP()

<IP  |>

In [ ]:
IP()/TCP()

In [ ]:
Ether()/IP()/TCP()

In [ ]:
IP()/TCP()/"GET / HTTP/1.0\r\n\r\n"

In [ ]:
Ehter()/IP()/IP()/UDP()

In [24]:
IP(proto=55)/TCP(dport=22)

<IP  frag=0 proto=55 |<TCP  dport=ssh |>>

In [ ]:
tcpip = IP(dst="dstIP") / TCP(dport=22) # tcp packet needs dest port
reply = sr(tcpip)
reply

In [ ]:
ans, unans = _ 
ans.summary()

## Create ping request
- IP packet with ICMP echo request and "Hello World" payload
- send the packet and capture it with wireshark

In [3]:
ping = IP(dst="172.16.7.111", src="172.16.7.129") / ICMP() / "Hello World!"

In [4]:
ping.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 172.16.7.129
  dst       = 172.16.7.111
  \options   \
###[ ICMP ]###
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
###[ Raw ]###
        load      = 'Hello World!'


In [5]:
send(ping)


Sent 1 packets.


In [ ]:
reply = sr1(ping)
reply.show()

## Exercise
- Spoof src IP and ping some IP in ACM or CSC network
- 